In [9]:
import os
import pandas as pd
from dotenv import load_dotenv
from ragas import evaluate
import base64


In [4]:
load_dotenv()

True

In [35]:
Rag_result = open('C:\Code\III_2024_golf\data\Rag_Eval_data.csv', 'r').read()
Rag_result_df = pd.read_csv('C:\Code\III_2024_golf\data\Rag_Eval_data.csv')
Rag_questions_df = pd.read_csv('C:\Code\III_2024_golf\data\Rag_question_Input.csv')
Rag_questions_df

,num,Input,GroundTruth
0,186382,Pull Hook 左拉左曲球\n球速:66.43、發射角度:7.198、發射方向:-6.1...,P4 頂點/轉換\n肩膀往右轉動\n桿頭頂點過高
1,186387,Fade 小右曲球\n球速:70.33、發射角度:9.56、發射方向:1.486、飛行距離:...,P4 頂點/轉換\n優秀的觸球！\n觸球時桿面角度方正
2,186410,Push Slice 右拉右曲球\n球速:61.868、發射角度:17.004、發射方向:1...,P2~3 上桿\n上桿(P2~3)時，左手腕維持固定，腰部減少轉動\n上桿(P2~3)時，角...
3,186416,Pull Hook 左拉左曲球\n球速:88.875、發射角度:7.848、發射方向:-7....,P7 Impact 擊球\n擊球時，左手腕應恢復成未彎曲的狀態，及注意擊球點位置\n桿面關閉...
4,189514,Push Slice 右拉右曲球\n球速:100.069、發射角度:17.951、發射方向:...,P7 Impact 擊球\n擊球時，左手腕維持固定，注意擊球點位置，置球點到擊球落點的距離達...
5,199353,Push Slice 右拉右曲球\n球速:72.111、發射角度:12.173、發射方向:6...,P5~6 下桿\n下桿時，手腕及肩膀往左轉動，腰部減少旋轉\n下桿角度過於平緩，左手腕過度外...


In [11]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [61]:
#移除所有answers中括弧內的文字
def remove_brackets(text):
    while text.find('[') != -1:
        start = text.find('[')
        end = text.find(']')
        text = text[:start] + text[end+1:]
    return text

In [64]:
from datasets import Dataset

questions = Rag_questions_df['Input'].tolist()
ground_truths = Rag_questions_df['GroundTruth'].tolist()
answers_NoRAG = Rag_result_df['NoRAG'].tolist()
answers_SimpleRAG = Rag_result_df['SimpleRAG'].tolist()
answers_GraphRAG = Rag_result_df['GraphRAG'].tolist()
awswer_HybridRAG = Rag_result_df['HybridRAG'].tolist()

for i,text in enumerate(answers_GraphRAG):
    answers_GraphRAG[i] = remove_brackets(text)
for i,text in enumerate(awswer_HybridRAG):
    awswer_HybridRAG[i] = remove_brackets(text)

dataset_NoRAG = Dataset.from_dict({
    "question": questions,
    "answer": answers_NoRAG,
    "ground_truths": ground_truths
})

dataset_SimpleRAG = Dataset.from_dict({
    "question": questions,
    "answer": answers_SimpleRAG,
    "ground_truths": ground_truths
})
dataset_GraphRAG = Dataset.from_dict({
    "question": questions,
    "answer": answers_GraphRAG,
    "ground_truths": ground_truths
})
dataset_HybridRAG = Dataset.from_dict({
    "question": questions,
    "answer": awswer_HybridRAG,
    "ground_truths": ground_truths
})


In [65]:
answers_GraphRAG[0]

'\n    "球路": "Pull Hook 左拉左曲球",\n    "原因": "您的揮桿出現了Pull Hook（左拉左曲球），主要原因是您的上桿（P2~3）角度過於陡峭，導致桿頭在頂點位置過高，進而影響下桿階段（P5~6），使得下桿角度過於陡峭。此外，您的左手腕在P4頂點轉換時過度彎曲，導致桿面閉合，並且揮桿路徑過度由內向外，這些因素共同導致擊球時桿面關閉，進一步加劇了球的左旋曲線 。",\n    "建議": "在我們全面的揮桿分析中，您的揮桿動作在P4頂點轉換時與標準動作有明顯的差異，您的左手腕過度彎曲，使得桿面過於閉合。因此，請在上桿至P3時，讓肩膀更自然地向右旋轉，減少桿頭上升過高的情況，以確保更平順的轉換 。\\n\\n同時，在下桿階段（P5~6），請專注於左手臂保持打直，讓左手腕維持固定，避免過度翻轉桿面，這將有助於減少擊球時的桿面閉合問題，提升擊球穩定性 。\\n\\n此外，您的下桿路徑過度由內向外，請嘗試在下桿時從臀部帶動揮桿，保持下半身與上半身的協調，避免過度使用手腕來調整球路。這樣的調整將有助於減少左飛和左曲的問題，使擊球更穩定 。\\n\\n透過這些技術調整，您的擊球將更精準，並有效降低左曲球的發生率 。"\n'

# Evaluate the RAG model

In [77]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,

)

output_df = pd.DataFrame()

dataset_name = ['dataset_NoRAG', 'dataset_SimpleRAG', 'dataset_GraphRAG', 'dataset_HybridRAG']

for i,dataset in enumerate([dataset_NoRAG, dataset_SimpleRAG, dataset_GraphRAG, dataset_HybridRAG]):
    result = evaluate(
        dataset = dataset, 
        metrics=[
            answer_relevancy,
        ],
    ).to_pandas()
    #change the column name
    result.columns = [f'{dataset_name[i]}_{col}' for col in result.columns]
    output_df = pd.concat([output_df, result], axis=1)
output_df.to_csv('C:\Code\III_2024_golf\data\Rag_Eval_result.csv', index=False)


Evaluating: 100%|██████████| 6/6 [00:05<00:00,  1.01it/s]


In [78]:
output_df

,dataset_NoRAG_user_input,dataset_NoRAG_response,dataset_NoRAG_answer_relevancy,dataset_SimpleRAG_user_input,dataset_SimpleRAG_response,dataset_SimpleRAG_answer_relevancy,dataset_GraphRAG_user_input,dataset_GraphRAG_response,dataset_GraphRAG_answer_relevancy,dataset_HybridRAG_user_input,dataset_HybridRAG_response,dataset_HybridRAG_answer_relevancy
0,Pull Hook 左拉左曲球\n球速:66.43、發射角度:7.198、發射方向:-6.1...,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因""...",0.746644,Pull Hook 左拉左曲球\n球速:66.43、發射角度:7.198、發射方向:-6.1...,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因""...",0.739472,Pull Hook 左拉左曲球\n球速:66.43、發射角度:7.198、發射方向:-6.1...,"\n ""球路"": ""Pull Hook 左拉左曲球"",\n ""原因"": ""您的揮...",0.754046,Pull Hook 左拉左曲球\n球速:66.43、發射角度:7.198、發射方向:-6.1...,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因"": ""下桿角度過...",0.746439
1,Fade 小右曲球\n球速:70.33、發射角度:9.56、發射方向:1.486、飛行距離:...,"""球路"": ""Fade 小右曲球"",\n ""原因"": ""整體...",0.721654,Fade 小右曲球\n球速:70.33、發射角度:9.56、發射方向:1.486、飛行距離:...,"""球路"": ""Fade 小右曲球"",\n ""原因"": ""手腕角...",0.725226,Fade 小右曲球\n球速:70.33、發射角度:9.56、發射方向:1.486、飛行距離:...,"""球路"": ""Fade 小右曲球"",\n ""原因"": ""整體揮桿姿勢與教練相差...",0.733188,Fade 小右曲球\n球速:70.33、發射角度:9.56、發射方向:1.486、飛行距離:...,"""球路"": ""Fade 小右曲球"",\n ""原因"": ""手腕角度保持得很好，桿...",0.742562
2,Push Slice 右拉右曲球\n球速:61.868、發射角度:17.004、發射方向:1...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因...",0.732604,Push Slice 右拉右曲球\n球速:61.868、發射角度:17.004、發射方向:1...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""P7擊球...",0.759734,Push Slice 右拉右曲球\n球速:61.868、發射角度:17.004、發射方向:1...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""P5~6...",0.695274,Push Slice 右拉右曲球\n球速:61.868、發射角度:17.004、發射方向:1...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""P7擊球...",0.753224
3,Pull Hook 左拉左曲球\n球速:88.875、發射角度:7.848、發射方向:-7....,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因...",0.755183,Pull Hook 左拉左曲球\n球速:88.875、發射角度:7.848、發射方向:-7....,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因""...",0.750884,Pull Hook 左拉左曲球\n球速:88.875、發射角度:7.848、發射方向:-7....,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因"": ""您的擊球呈...",0.749375,Pull Hook 左拉左曲球\n球速:88.875、發射角度:7.848、發射方向:-7....,"""球路"": ""Pull Hook 左拉左曲球"",\n ""原因"": ""下桿時桿面...",0.746114
4,Push Slice 右拉右曲球\n球速:100.069、發射角度:17.951、發射方向:...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原...",0.753686,Push Slice 右拉右曲球\n球速:100.069、發射角度:17.951、發射方向:...,"""球路"": ""Push Slice右拉右曲球"",\n ""原因""...",0.770707,Push Slice 右拉右曲球\n球速:100.069、發射角度:17.951、發射方向:...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""您的上桿...",0.760462,Push Slice 右拉右曲球\n球速:100.069、發射角度:17.951、發射方向:...,"""球路"": ""Push Slice右拉右曲球"",\n ""原因"": ""P5~6下...",0.767070
5,Push Slice 右拉右曲球\n球速:72.111、發射角度:12.173、發射方向:6...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因...",0.735840,Push Slice 右拉右曲球\n球速:72.111、發射角度:12.173、發射方向:6...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""P5~6下桿角度...",0.762893,Push Slice 右拉右曲球\n球速:72.111、發射角度:12.173、發射方向:6...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""根據揮桿...",0.729027,Push Slice 右拉右曲球\n球速:72.111、發射角度:12.173、發射方向:6...,"""球路"": ""Push Slice 右拉右曲球"",\n ""原因"": ""P5~6下桿角度...",0.755902


In [79]:
# 計算answer_relevancy平均值
output_df[['dataset_NoRAG_answer_relevancy', 'dataset_SimpleRAG_answer_relevancy', 'dataset_GraphRAG_answer_relevancy', 'dataset_HybridRAG_answer_relevancy']].mean()

dataset_NoRAG_answer_relevancy        0.740935
dataset_SimpleRAG_answer_relevancy    0.751486
dataset_GraphRAG_answer_relevancy     0.736895
dataset_HybridRAG_answer_relevancy    0.751885
dtype: float64